In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget
%pdb off

from pyCascade.cti import lesCreateImage as lcm
from pyCascade.cti import lesCreateMovie as vcm
import os
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np
import pandas as pd
import cti_image
import seaborn as sns

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)

In [4]:
multiRun_dir = "CHARLES/multiRuns"
pcStatsMI = pd.read_csv(f"{multiRun_dir}/pointCloudStatsNoIntMI.csv", index_col=[0,1])
pcStatsMI.columns = pcStatsMI.columns.str.replace("-noInt", "")
pcStatsMI = pcStatsMI.droplevel(0)
pcStatsMI["run"] = pcStatsMI["run"] // 10 * 10
pcStatsMI = pcStatsMI.drop_duplicates()
pcStatsMI = pcStatsMI.reset_index()
pcStatsMI = pcStatsMI.set_index(["run", "index"])

pcStatsMI = pcStatsMI[pcStatsMI["houseType"]!="-1-0"]

In [5]:
pcStatsMI = pcStatsMI[pcStatsMI["openingType"] != "skylight"]

In [ ]:
def getImagePath(row, qoi, step = 119000):
    # get dir
    run_Id = str(row.name[0])
    config = int(run_Id[0])
    run = int(run_Id[1:3])
    imageDir = f"{scratch_dir}/CHARLES/config{config}/R{run}/Images"
    # get suffix
    step = str(step)
    while len(step) < 8:
        step = "0" + str(step)
    blockType = row["blockType"]
    if blockType == "B":
        blockType = ""
    else:
        blockType = f"_{blockType}"
    imageSuffix = f"h_{row['houseType']}{blockType}.{step}.png"
    # full path
    imagePath = f"{imageDir}/{qoi}_{imageSuffix}"
    return imagePath
    
def find_nearest_valid_pixel(im_zone, i, j, max_radius=10):
    h, w = im_zone.shape
    candidates = []

    for di in range(-max_radius, max_radius + 1):
        for dj in range(-max_radius, max_radius + 1):
            ni, nj = i + di, j + dj
            if 0 <= ni < h and 0 <= nj < w:
                if im_zone[ni, nj] >= 100:
                    distance = np.hypot(di, dj)  # Euclidean distance
                    candidates.append(((ni, nj), distance))

    if not candidates:
        return None  # nothing found within radius

    # Return the pixel with the smallest Euclidean distance
    candidates.sort(key=lambda x: x[1])
    return candidates[0][0]

def getImageData(row, qoi, verbose=False):
    if verbose: print(row.name)
    imagePath = getImagePath(row, qoi)
    if verbose: print(imagePath)

    x = row["x"]
    y = row["y"]
    z = row["z"]
    if verbose: print(f"x = {x}, y = {y}, z = {z}")

    im = cti_image.Image(imagePath)
    im_ijk = im.getImgCoordForSim(x, y, z)

    im_i = im_ijk[1]
    im_j = im_ijk[0]
    if verbose: print(f"i = {im_i}, j = {im_j}, initial zoNe = {im.zoNe[im_i, im_j]}")

    pixel = find_nearest_valid_pixel(im.zoNe, im_i, im_j)
    if pixel is None:
        if verbose: print("No valid pixel found within the max radius.")
        return np.nan, np.nan
    else:
        im_i, im_j = pixel
    if verbose: print(f"i = {im_i}, j = {im_j}, valid zoNe = {im.zoNe[im_i, im_j]}")


    sim_coord = im.getSimCoordsForIJ(im_i, im_j)
    if verbose: print(f"sim coord = {sim_coord}")

    imagePixelDist = np.sqrt((x-sim_coord[0])**2 + (y-sim_coord[1])**2 + (z-sim_coord[2])**2)
    if verbose: print(f"Distance to found pixel: {imagePixelDist}")

    var_min = im.metadata['planar']['range'][0]
    var_max = im.metadata['planar']['range'][1]

    data_gs = np.mean(im.getRGB()[im_i, im_j], axis=-1) / 255
    data_scaled = data_gs * (var_max - var_min) + var_min # scale up with local range

    return data_scaled, imagePixelDist

getImageData(pcStatsMI.iloc[0], "p_avg", verbose=True)
    

In [ ]:
# pcStatsMI["pImage"], pcStatsMI["distPixel"] = pcStatsMI.apply(lambda row: getImageData(row, "p_avg"), axis=1)
imData = pcStatsMI.apply(lambda row: getImageData(row, "p_avg"), axis=1)

In [ ]:
pcStatsMI.loc[:, "p_img"] = imData.apply(lambda x: x[0])
pcStatsMI.loc[:, "dist_pixel"] = imData.apply(lambda x: x[1])
pcStatsMI

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=pcStatsMI, x="p_img", y="p_avg", hue="dist_pixel")